In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


In [14]:
train_dir = '/kaggle/input/100-bird-species/train'
test_dir = '/kaggle/input/100-bird-species/test'
validation_dir = '/kaggle/input/100-bird-species/valid'

# Define the image size and batch size
image_size = (150, 150)
batch_size = 32

In [15]:
# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescaling for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Loading the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [16]:
from tensorflow.keras.applications import MobileNet

In [24]:
num_classes = len(os.listdir(os.path.join(path, 'train')))
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

for layer in base_model.layers:
    layer.trainable = False

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

Epoch 1/10
2644/2644 [==============================] - 1466s 549ms/step - loss: 2.6909 - accuracy: 0.4040 - val_loss: 1.3652 - val_accuracy: 0.6319
Epoch 2/10
2644/2644 [==============================] - 788s 298ms/step - loss: 1.5412 - accuracy: 0.6101 - val_loss: 1.0664 - val_accuracy: 0.7134
Epoch 3/10
2644/2644 [==============================] - 778s 294ms/step - loss: 1.3689 - accuracy: 0.6490 - val_loss: 1.0211 - val_accuracy: 0.7165
Epoch 4/10
2644/2644 [==============================] - 769s 291ms/step - loss: 1.2886 - accuracy: 0.6683 - val_loss: 0.9914 - val_accuracy: 0.7287
Epoch 5/10
2644/2644 [==============================] - 791s 299ms/step - loss: 1.2390 - accuracy: 0.6800 - val_loss: 0.9592 - val_accuracy: 0.7443
Epoch 6/10
2644/2644 [==============================] - 810s 306ms/step - loss: 1.2078 - accuracy: 0.6860 - val_loss: 0.9365 - val_accuracy: 0.7428
Epoch 7/10
2644/2644 [==============================] - 846s 320ms/step - loss: 1.1852 - accuracy: 0.6924 - val

In [29]:
y_pred = model.predict(test_generator)

83/83 [==============================] - 6s 76ms/step


In [66]:
img = plt.imread(os.path.join(path, 'test', 'DALMATIAN PELICAN', '5.jpg'))
img = cv2.resize(img, (224, 224))

In [67]:
out = model.predict(np.expand_dims(img, axis=0))

1/1 [==============================] - 0s 22ms/step


In [68]:
classes = os.listdir(os.path.join(path, 'train'))
classes[out.argmax()]

'RED TAILED HAWK'

In [71]:
np.save('classes.npy', classes)

In [69]:
model.save('birb-classifier.h5')

In [78]:
pred_fn(img)

1/1 [==============================] - 0s 22ms/step


'RED TAILED HAWK'